# 3 Planning and Bitcoin opening values Report Generation

## Setup

In [2]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-4o", "api_key":OPENAI_API_KEY, "temperature":1.0,"cache_seed": None}

from autogen import ConversableAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen.coding import LocalCommandLineCodeExecutor

## The task!

In [ ]:
# Task definition
task = "Generate a report about the last-20-days bitcoin opening price as of March 04, 2025. "\
       "Use the yfinance, matplotlib and datetime python packages , "\
        "and save the plot of the trend of the bitcoin opening value as image."\
        "The report must be written by a writer expert."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Code executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

Create a directory to store the code files.

In [ ]:
code_dir = "./code2/"

Define the agents

In [ ]:
# Agent acting on behalf of the user (-->"Human in the loop!")
user_proxy = ConversableAgent(
    name="Admin",
    system_message="Provide the task and give feedback or instructions to other agents.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Agent that plans the steps necessary to complete the task
planner = ConversableAgent(
    name="Planner",
    system_message="Analyze the task and determine the steps and information needed "
                   "to complete it using yfinance,matplotlib and datetime python packages. Guide the team "
                   "through the process."
                   "Assume all the packages are already installed."
                   "You cannot write python code.",
    llm_config=llm_config,
    description="Plans the steps and coordinates the workflow."
)

# Agent engineer specialized in writing python code
engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="Write Python code based on the planner's instructions. "
                   "Use the 'yfinance' Python package to get data about bitcoin opening price value of the last 20 days,"
                    "considering that today is 2025-04-03."
                    "Assume that all the necessary python packages are already installed."
                    "Print the dictionary containing as key the date as a string and the corresponding opening price as value.",
    description="Writes the necessary code to fetch and analyze data with yfinance, datetime and matplotlib."
)


# Create a local command line code executor.
# It saves and executes LLM generated python code
executor = LocalCommandLineCodeExecutor(
    timeout=60,  # Timeout for each code execution in seconds.
    work_dir=code_dir
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    system_message="Execute the code provided by the Engineer and report the results or errors.",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    description="Runs the code using a local command-line executor."

)


# Agent specialized in writing report
writer = ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Write a markdown report based on the executed code results. "
                   "Format the report with a title and content in ```md``` blocks. "
                   "Refine the report based on Admin feedback.",
    description="Creates the final report about the trend of the last 20 days bitcoin opening values."
)


# Define the group chat
# This is like a chat room where multiple agents share one conversation thread. 
# They collaborate on a task, and all agents see the same context. 
# You set it up with a list of agents and rules, 
# like how many rounds of chatting are allowed or how the next speaker is chosen (e.g., randomly or automatically).
groupchat = GroupChat(
    agents=[user_proxy, planner, engineer, code_executor_agent, writer],
    messages=[],
    max_round=15,
    speaker_selection_method="auto" # round_robin, random, manual
)

# Create the group chat manager
# This is the "organizer" of the GroupChat. It’s a special agent that controls the flow, 
# deciding which agent speaks next based on the conversation so far. 
# It uses a language model (like GPT) to pick the next speaker and keeps the chat moving until the task is done or a limit is reached.
manager = GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

# Start the group chat
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

# Optional: Print the result (for debugging or inspection)
print(groupchat_result)

Admin (to chat_manager):

Generate a report about the last-20-days bitcoin opening price as of March 04, 2025. Use the yfinance, matplotlib and datetime python packages , and save the plot of the trend of the bitcoin opening value as image.The report must be written by a writer expert.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To generate a report about the last-20-days Bitcoin opening price as of March 04, 2025, you can follow these steps:

1. **Import Necessary Modules**:
   - Begin by importing the necessary modules: `yfinance` to fetch the financial data, `matplotlib` to create the plot, and `datetime` to work with the date and time.

2. **Determine the Date Range**:
   - Utilize the `datetime` module to calculate the date range. Specifically, get the last 20 trading days leading up to March 04, 2025.

3. **Fetch Bitcoin Data Using yfinance**:
   - Use `yfinance

In [ ]:
groupchat_result.chat_history

ChatResult(chat_id=None, chat_history=[{'content': 'Generate a report about the last-20-days bitcoin opening price, using the yfinance,matplotlib and datetime python packages as of March 04, 2025, and save the plot of the trend of the bitcoin opening value as image.', 'role': 'assistant', 'name': 'Admin'}, {'content': "To generate a report about the last 20 days of Bitcoin opening prices as of March 04, 2025, follow these steps:\n\n1. **Import Necessary Modules**:\n   - Ensure the `yfinance`, `matplotlib.pyplot`, and `datetime` libraries are available and import them into your script.\n\n2. **Define the Dates**:\n   - Use the `datetime` module to define the end date as March 04, 2025.\n   - Calculate the start date by subtracting 19 days from the end date to cover the last 20 days (including the end date).\n\n3. **Download Stock Data**:\n   - Use the `yfinance` library to download Bitcoin price data. The ticker symbol for Bitcoin is typically 'BTC-USD'.\n   - Set the start date and end